In [284]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from itertools import combinations
from tqdm.auto import tqdm
from collections import deque
from operator import itemgetter

In [92]:
vanuatu = pd.read_csv('cities/vanuatu.csv', header=None, names=['city', 'latitude', 'longitude'])
italy = pd.read_csv('cities/italy.csv', header=None, names=['city', 'latitude', 'longitude'])
russia = pd.read_csv('cities/russia.csv', header=None, names=['city', 'latitude', 'longitude'])
us = pd.read_csv('cities/us.csv', header=None, names=['city', 'latitude', 'longitude'])
china = pd.read_csv('cities/china.csv', header=None, names=['city', 'latitude', 'longitude'])

In [289]:
def compute_distance(city1, city2):
    return geodesic(
        (city1.latitude, city1.longitude), (city2.latitude, city2.longitude)
        ).km

## Greedy algorithm

In [180]:
def create_dist_matrix(cities):
    dist_matrix = np.zeros((cities.shape[0], cities.shape[0]))
    for c1, c2 in combinations(cities.itertuples(), 2):
        dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = compute_distance(c1,c2)
    return dist_matrix

def greedy_tsp(cities):
    num_cities = cities.shape[0]
    dist_matrix = create_dist_matrix(cities)
    route = []
    current_city_index = 0
    route.append(current_city_index)
    cost = 0

    while not len(route)==num_cities:
        dist_matrix[:, current_city_index] = np.inf
        closest_city_index = np.argmin(dist_matrix[current_city_index])
        route.append(closest_city_index)
        cost += dist_matrix[current_city_index, closest_city_index]
        current_city_index = closest_city_index
        
    cost += compute_distance(cities.iloc[route[-1]], cities.iloc[route[0]])
    route.append(route[0])    
    return route, cost

In [182]:
print(greedy_tsp(vanuatu))

([0, np.int64(7), np.int64(1), np.int64(4), np.int64(3), np.int64(5), np.int64(2), np.int64(6), 0], np.float64(1475.528091104531))


In [290]:
print(greedy_tsp(italy)[1])

4436.03176952516


In [97]:
print(greedy_tsp(russia)[1])

42334.16465744784


In [98]:
print(greedy_tsp(us)[1])

48050.02586446137


In [99]:
print(greedy_tsp(china)[1])

63962.9184294552


## Greedier algorithm

In [176]:
def greedier_tsp(cities):

    min_cost = np.inf
    min_route = None
    num_cities = cities.shape[0]
    max_iter = np.min((num_cities, 15))

    for _ in tqdm(range(max_iter)):
        dist_matrix = create_dist_matrix(cities)
        route = []
        current_city_index = np.random.randint(num_cities)
        route.append(current_city_index)
        cost = 0

        while not len(route)==num_cities:
            dist_matrix[:, current_city_index] = np.inf
            closest_city_index = np.argmin(dist_matrix[current_city_index])
            route.append(closest_city_index)
            cost += dist_matrix[current_city_index, closest_city_index]
            current_city_index = closest_city_index
        
        cost += compute_distance(cities.iloc[route[-1]], cities.iloc[route[0]])
        route.append(route[0])

        if cost < min_cost:
            min_cost = cost
            min_route = route


    return min_route, min_cost

In [177]:
print(greedier_tsp(vanuatu))

  0%|          | 0/8 [00:00<?, ?it/s]

([0, np.int64(7), np.int64(1), np.int64(4), np.int64(3), np.int64(5), np.int64(2), np.int64(6), 0], np.float64(1475.528091104531))


In [291]:
print(greedier_tsp(italy))

  0%|          | 0/15 [00:00<?, ?it/s]

([33, np.int64(12), np.int64(30), np.int64(9), np.int64(4), np.int64(19), np.int64(32), np.int64(25), np.int64(28), np.int64(18), np.int64(20), np.int64(3), np.int64(6), np.int64(44), np.int64(45), np.int64(23), np.int64(43), np.int64(41), np.int64(5), np.int64(40), np.int64(22), np.int64(42), np.int64(13), np.int64(16), np.int64(29), np.int64(10), np.int64(26), np.int64(39), np.int64(34), np.int64(15), np.int64(14), np.int64(21), np.int64(35), np.int64(11), np.int64(1), np.int64(2), np.int64(38), np.int64(17), np.int64(31), np.int64(8), np.int64(37), np.int64(24), np.int64(7), np.int64(36), np.int64(27), np.int64(0), 33], np.float64(4436.03176952516))


In [167]:
print(greedier_tsp(russia)[1])

41526.35101041394


In [168]:
print(greedier_tsp(us)[1])

47538.6574768417


In [169]:
# print(greedier_tsp(china)[1])
# This computation took 12m36s on my machine. Computationally heavy and totally not optimal for large set of cities

63057.19111098851


## Tabu Search

In [221]:
def fitness(solution, dist_matrix):
    total_distance = 0
    for i in range(len(solution) - 1):
        city1 = solution[i]
        city2 = solution[i + 1]
        total_distance += dist_matrix[city1][city2]
        
    return total_distance

In [216]:
def inversion_mutation(solution):
    pos1 = np.random.randint(0, len(solution)-2)
    pos2 = np.random.randint(pos1, len(solution)-2)
    
    new_solution = solution.copy()
    new_solution.pop()
    new_solution[pos1:pos2+1] = new_solution[pos1:pos2+1][::-1]
    new_solution.append(new_solution[0])
    return new_solution

In [215]:
def scramble_mutation(solution):
    pos1 = np.random.randint(0, len(solution)-2)
    pos2 = np.random.randint(pos1, len(solution)-2)
    
    new_solution = solution.copy()
    new_solution.pop()
    subsequence = new_solution[pos1:pos2+1]
    np.random.shuffle(subsequence)
    new_solution[pos1:pos2+1] = subsequence
    new_solution.append(new_solution[0])
    return new_solution

In [263]:
def generate_neighbor(solution):
    if np.random.random() < 0.8:
        return inversion_mutation(solution)
    return scramble_mutation(solution)

In [264]:
def tabu_search(init_solution, dist_matrix, tabu_memory=10, max_worsening_moves=5, max_iterations=1000):

    current_solution = init_solution.copy()
    best_solution = init_solution.copy()
    current_fitness = fitness(current_solution, dist_matrix)
    best_fitness = current_fitness
    
    tabu_list = deque(maxlen=tabu_memory)
    tabu_list.append(tuple(current_solution))
    
    worsening_moves = 0
    iteration = 0
    
    while iteration < max_iterations:
        neighbor = generate_neighbor(current_solution)
        neighbor_fitness = fitness(neighbor, dist_matrix)
        
        if neighbor_fitness < best_fitness:
            best_solution = neighbor.copy()
            best_fitness = neighbor_fitness
            worsening_moves = 0
        else:
            worsening_moves += 1
        
        # accept neighbor if not tabu or if it's better than best known
        if tuple(neighbor) not in tabu_list or neighbor_fitness < best_fitness:
            current_solution = neighbor
            current_fitness = neighbor_fitness
            tabu_list.append(tuple(neighbor))
        
        # if too many worsening moves, return to best solution
        if worsening_moves >= max_worsening_moves:
            current_solution = best_solution.copy()
            current_fitness = best_fitness
            worsening_moves = 0
        
        iteration += 1
    
    return best_solution, best_fitness

In [267]:
cities = italy
dist_matrix = create_dist_matrix(cities)
init_solution, init_cost = greedy_tsp(cities)


print(f"Initial distance: {init_cost}")
print(f"Distance of the route found with taboo: {tabu_search(init_solution, dist_matrix)[1]}")

Initial distance: 4436.03176952516
Distance of the route found with taboo: 4329.671303083085


## ES (Evolved Salesman)

In [293]:
def create_initial_population(init_solution, pop_size):
    population = [init_solution.copy()]
    
    while len(population) < pop_size:
        if np.random.random() < 0.5:
            new_solution = inversion_mutation(init_solution)
        else:
            new_solution = scramble_mutation(init_solution)
        population.append(new_solution)
    
    return population

In [312]:
def rank_based_selection(population, fitness_values, num_parents):
    # lower fitness -> higher rank
    population_fitness = list(zip(population, fitness_values))
    population_fitness.sort(key=itemgetter(1))
    
    n = len(population_fitness)
    ranks = range(n, 0, -1)
    rank_sum = sum(ranks)
    probabilities = [r/rank_sum for r in ranks]
    
    selected_indices = np.random.choice(n, size=num_parents, p=probabilities, replace=False)
    
    return [population_fitness[i][0] for i in selected_indices]

In [313]:
def inver_over_crossover(parent1, parent2):
    p1 = parent1[:-1].copy()
    p2 = parent2[:-1].copy()
    offspring = p1.copy()
    
    position1 = np.random.randint(0, len(p1)-1)
    current_element = p1[position1]
    next_element = p1[(position1+1) % len(p1)]

    p2_current_pos = p2.index(current_element)
    p2_next_element = p2[(p2_current_pos + 1) % len(p2)]

    if p2_next_element == next_element:
        p2_next_element = p2[(p2_current_pos + 2) % len(p2)]
    
    position2 = p1.index(p2_next_element)
    
    #determine the inversion section
    if position1 < position2:
        offspring[position1+1:position2+1] = offspring[position1+1:position2+1][::-1]
    else:
        temp = offspring[position1+1:] + offspring[:position2+1]
        temp = temp[::-1]
        offspring[position1+1:] = temp[:len(offspring)-position1-1]
        offspring[:position2+1] = temp[len(offspring)-position1-1:]
    
    offspring.append(offspring[0])  
    return offspring

In [305]:
def evolution_strategy(init_solution, dist_matrix, pop_size=50, offspring_size=10, 
                      num_generations=1000, mutation_prob=0.2):
    population = create_initial_population(init_solution, pop_size)
    fitness_values = [fitness(sol, dist_matrix) for sol in population]
    best_solution = min(zip(population, fitness_values), key=itemgetter(1))[0]
    best_fitness = min(fitness_values)
    fitness_history = [best_fitness]

    for generation in tqdm(range(num_generations)):
        for _ in range(offspring_size):
            parents = rank_based_selection(population, fitness_values, 2)
            offspring = inver_over_crossover(parents[0], parents[1])

            if np.random.random() < mutation_prob:
                if np.random.random() < 0.7:
                    offspring = inversion_mutation(offspring)
                else:
                    offspring = scramble_mutation(offspring)
            
            offspring_fitness = fitness(offspring, dist_matrix)
            
            #compare offspring with the worst solution in the population
            worst_index = max(range(len(fitness_values)), key=lambda i: fitness_values[i])
            if offspring_fitness < fitness_values[worst_index]:
                population[worst_index] = offspring
                fitness_values[worst_index] = offspring_fitness
                
                if offspring_fitness < best_fitness:
                    best_fitness = offspring_fitness
                    best_solution = offspring.copy()
        
        fitness_history.append(best_fitness)
    
    return best_solution, best_fitness, fitness_history

In [299]:
cities = italy
dist_matrix = create_dist_matrix(cities)
init_solution, init_cost = greedy_tsp(cities)


print(f"Initial distance: {init_cost}")
print(f"Distance of the route found with ES: {evolution_strategy(init_solution, dist_matrix)[1]}")

Initial distance: 4436.03176952516


  0%|          | 0/1000 [00:00<?, ?it/s]

Distance of the route found with ES: 4260.041049398426


In [301]:
cities = china
dist_matrix = create_dist_matrix(cities)
init_solution, init_cost = greedy_tsp(cities)


print(f"Initial distance: {init_cost}")
print(f"Distance of the route found with ES: {evolution_strategy(init_solution, dist_matrix, num_generations=100000)[1]}")

Initial distance: 63962.9184294552


  0%|          | 0/100000 [00:00<?, ?it/s]

Distance of the route found with ES: 58331.58041096962
